In [86]:
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import numpy as np

import sys
sys.path.append('./../scripts')

from refuelplot import *
setup()
sns.set_style("darkgrid")

In [87]:
tx_path = '/data/projects/texas-power-outages/data/interim/'
fig_path = '/data/projects/texas-power-outages/data/figures/'
out_path = '/data/projects/texas-power-outages/data/output/'
temp_path = '/data/projects/texas-power-outages/data/input/temperatures/'
gf_path = '/data/projects/texas-power-outages/data/input/gas_production/'
ct_path = '/data/projects/texas-power-outages/data/input/shp/'
pp_path = '/data/projects/texas-power-outages/data/input/powerplants/'
wind_path = '/data/users/kgruber/Data/era5/USA/eff_ws/'
var_path = '/data/users/kgruber/Data/era5/TX/'

In [88]:
# get temperature
temp = xr.open_mfdataset(temp_path + 'era5_temp_TX_??????.nc').t2m-273.15

# temperatures with outage power plants

In [8]:
outages = pd.read_feather(tx_path + 'outages/outages.feather')
outages = outages[outages.dataset=='edgar'].drop('dataset',axis=1)

/home/kgruber/.conda/envs/py37/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


## gas

### all

In [50]:
outagesNG = outages[(outages.fuel=='NG')].set_index('ts').resample('h').sum()

In [10]:
lonlatNG = outages[outages.fuel=='NG'][['Latitude','Longitude','cap_max']].drop_duplicates(subset=['Latitude','Longitude'],keep='first')
startNG = outages[(outages.fuel=='NG')&(outages.reduction>0)].drop_duplicates(subset=['Latitude','Longitude'],keep='first').ts
noavailNG = outages[(outages.fuel=='NG')&(outages.cap_available==0)].drop_duplicates(subset=['Latitude','Longitude'],keep='first').ts

In [51]:
tempNG = temp.interp(coords={"longitude":xr.DataArray(lonlatNG.Longitude.values,dims='location'),
                              "latitude":xr.DataArray(lonlatNG.Latitude.values,dims='location')},
                      method="nearest")

In [52]:
tgNGw = (tempNG*lonlatNG.cap_max.values/lonlatNG.cap_max.sum()).sum('location').to_dataframe()

In [53]:
tgNGw.to_csv('/data/projects/texas-power-outages/data/interim/temperatures/temp_gas_outage.csv')

### NS split

In [9]:
# split outages in North and South
latlimit = 30
outagesNGN = outages[(outages.fuel=='NG')&(outages.Latitude>latlimit)].set_index('ts').resample('h').sum()
outagesNGS = outages[(outages.fuel=='NG')&(outages.Latitude<=latlimit)].set_index('ts').resample('h').sum()

In [11]:
# split locations in North and South
latlimit = 30
lonlatNGN = lonlatNG[lonlatNG.Latitude>latlimit]
lonlatNGS = lonlatNG[lonlatNG.Latitude<=latlimit]

In [20]:
tempNGN = temp.interp(coords={"longitude":xr.DataArray(lonlatNGN.Longitude.values,dims='location'),
                              "latitude":xr.DataArray(lonlatNGN.Latitude.values,dims='location')},
                      method="nearest")
tempNGS = temp.interp(coords={"longitude":xr.DataArray(lonlatNGS.Longitude.values,dims='location'),
                              "latitude":xr.DataArray(lonlatNGS.Latitude.values,dims='location')},
                      method="nearest")

In [48]:
tgNGNw = (tempNGN*lonlatNGN.cap_max.values/lonlatNGN.cap_max.sum()).sum('location').to_dataframe()
tgNGSw = (tempNGS*lonlatNGS.cap_max.values/lonlatNGS.cap_max.sum()).sum('location').to_dataframe()

In [49]:
tgNGNw.to_csv('/data/projects/texas-power-outages/data/interim/temperatures/temp_gas_outageN.csv')
tgNGSw.to_csv('/data/projects/texas-power-outages/data/interim/temperatures/temp_gas_outageS.csv')

In [29]:
outagesNGN.to_csv('/data/projects/texas-power-outages/data/interim/outages/outagesNGN.csv')
outagesNGS.to_csv('/data/projects/texas-power-outages/data/interim/outages/outagesNGS.csv')

## coal

### all

In [50]:
outagesCOAL = outages[(outages.fuel=='COAL')].set_index('ts').resample('h').sum()

In [54]:
lonlatCOAL = outages[outages.fuel=='COAL'][['Latitude','Longitude','cap_max']].drop_duplicates(subset=['Latitude','Longitude'],keep='first')
startCOAL = outages[(outages.fuel=='COAL')&(outages.reduction>0)].drop_duplicates(subset=['Latitude','Longitude'],keep='first').ts
noavailCOAL = outages[(outages.fuel=='COAL')&(outages.cap_available==0)].drop_duplicates(subset=['Latitude','Longitude'],keep='first').ts

In [56]:
tempCOAL = temp.interp(coords={"longitude":xr.DataArray(lonlatCOAL.Longitude.values,dims='location'),
                              "latitude":xr.DataArray(lonlatCOAL.Latitude.values,dims='location')},
                      method="nearest")

In [57]:
tgCOALw = (tempCOAL*lonlatCOAL.cap_max.values/lonlatCOAL.cap_max.sum()).sum('location').to_dataframe()

In [58]:
tgCOALw.to_csv('/data/projects/texas-power-outages/data/interim/temperatures/temp_coal_outage.csv')

# temperatures with all power plants

In [59]:
pp = pd.read_csv(pp_path + 'PowerPlants_US_202004.csv')

## gas

### all

In [60]:
ppNG = pp[(pp.NG_MW>0)&(pp.StateName=='Texas')]

In [61]:
tppNG = temp.interp(coords={"longitude":xr.DataArray(ppNG.Longitude.values,dims='location'),
                             "latitude":xr.DataArray(ppNG.Latitude.values,dims='location')},
                     method="nearest")

In [104]:
# weigh by installed capacity and sum up
tempppNG = (tppNG*ppNG.NG_MW.values/ppNG.NG_MW.values.sum()).sum('location').to_dataframe()

In [105]:
tempppNG.to_csv('/data/projects/texas-power-outages/data/interim/temperatures/temp_gas_powerplant.csv')

### NS split

In [64]:
latlimit = 30
ppNGN = ppNG[ppNG.Latitude>latlimit]
ppNGS = ppNG[ppNG.Latitude<=latlimit]

In [65]:
tppNGN = temp.interp(coords={"longitude":xr.DataArray(ppNGN.Longitude.values,dims='location'),
                             "latitude":xr.DataArray(ppNGN.Latitude.values,dims='location')},
                     method="nearest")
tppNGS = temp.interp(coords={"longitude":xr.DataArray(ppNGS.Longitude.values,dims='location'),
                             "latitude":xr.DataArray(ppNGS.Latitude.values,dims='location')},
                     method="nearest")

In [106]:
# weigh by installed capacity and sum up
tempppNGN = (tppNGN*ppNGN.NG_MW.values/ppNGN.NG_MW.values.sum()).sum('location').to_dataframe()
tempppNGS = (tppNGS*ppNGS.NG_MW.values/ppNGS.NG_MW.values.sum()).sum('location').to_dataframe()

In [107]:
tempppNGN.to_csv('/data/projects/texas-power-outages/data/interim/temperatures/temp_gas_powerplantN.csv')
tempppNGS.to_csv('/data/projects/texas-power-outages/data/interim/temperatures/temp_gas_powerplantS.csv')

## coal

### all

In [71]:
ppCOAL = pp[(pp.Coal_MW>0)&(pp.StateName=='Texas')]

In [72]:
tppCOAL = temp.interp(coords={"longitude":xr.DataArray(ppCOAL.Longitude.values,dims='location'),
                             "latitude":xr.DataArray(ppCOAL.Latitude.values,dims='location')},
                     method="nearest")

In [108]:
# weigh by installed capacity and sum up
tempppCOAL = (tppCOAL*ppCOAL.Coal_MW.values/ppCOAL.Coal_MW.values.sum()).sum('location').to_dataframe()

In [109]:
tempppCOAL.to_csv('/data/projects/texas-power-outages/data/interim/temperatures/temp_coal_powerplant.csv')

## temperatures with gasfields

### all

In [75]:
# get gas fields
gasfields = pd.read_excel(gf_path + 'www.rrc.state.tx.us_media_qcpp3bau_2020-12-monthly-production-county-gas.xlsx').replace('DE WITT','DEWITT')

In [76]:
# get county centers
counties = pd.read_csv(ct_path + 'counties.csv')
counties = pd.DataFrame({'lon':counties.x.values,
                        'lat':counties.y.values},
                       index = counties.COUNTY.str.upper())

In [78]:
gfll = pd.DataFrame({'lon':counties.loc[gasfields.COUNTY,'lon'].values,
                     'lat':counties.loc[gasfields.COUNTY,'lat'].values,
                     'gp':gasfields['GAS-PRODUCTION-DECEMBRE-2020'].values,
                     'ct':gasfields['COUNTY'].values})

In [79]:
tgf = temp.interp(coords={"longitude":xr.DataArray(gfll.lon.values,dims='location'),
                              "latitude":xr.DataArray(gfll.lat.values,dims='location')},
                      method="nearest").assign_coords(location=gfll.ct.values)

In [110]:
# weigh by gas production and sum up
tempgfNG = (tgf*gfll.gp.values/gfll.gp.values.sum()).sum('location').to_dataframe()

In [111]:
tempgfNG.to_csv('/data/projects/texas-power-outages/data/interim/temperatures/temp_gasfields.csv')

### NS split

In [82]:
latlimit = 30
gfllN = gfll[gfll.lat>latlimit]
gfllS = gfll[gfll.lat<=latlimit]

In [83]:
tgfN = temp.interp(coords={"longitude":xr.DataArray(gfllN.lon.values,dims='location'),
                              "latitude":xr.DataArray(gfllN.lat.values,dims='location')},
                      method="nearest").assign_coords(location=gfllN.ct.values)
tgfS = temp.interp(coords={"longitude":xr.DataArray(gfllS.lon.values,dims='location'),
                              "latitude":xr.DataArray(gfllS.lat.values,dims='location')},
                      method="nearest").assign_coords(location=gfllS.ct.values)

In [112]:
# weigh by gas production and sum up
tempgfNGN = (tgfN*gfllN.gp.values/gfllN.gp.values.sum()).sum('location').to_dataframe()
tempgfNGS = (tgfS*gfllS.gp.values/gfllS.gp.values.sum()).sum('location').to_dataframe()

In [113]:
tempgfNGN.to_csv('/data/projects/texas-power-outages/data/interim/temperatures/temp_gasfieldsN.csv')
tempgfNGS.to_csv('/data/projects/texas-power-outages/data/interim/temperatures/temp_gasfieldsS.csv')

# trash

In [4]:
# get wind
wind = xr.open_mfdataset(wind_path + 'era5_wind_USA*.nc').wh10

In [5]:
# get dewpoint temperature
tempDP = xr.open_mfdataset(var_path + 'era5_tempDP_TX_??????.nc').d2m-273.15

In [6]:
# get precipitation
prec = xr.open_mfdataset(var_path + 'era5_prec_TX_??????.nc').tp

In [7]:
# get radiance
rad = xr.open_mfdataset(var_path + 'era5_rad_TX_??????.nc').ssr

In [13]:
tempi = temp.interp(coords={"longitude":xr.DataArray(lonlatNGN.Longitude.values,dims='location'),
                              "latitude":xr.DataArray(lonlatNGN.Latitude.values,dims='location')},
                      method="nearest")
tempDPi = tempDP.interp(coords={"longitude":xr.DataArray(lonlatNGN.Longitude.values,dims='location'),
                              "latitude":xr.DataArray(lonlatNGN.Latitude.values,dims='location')},
                      method="nearest")
tempiw = (tempi*lonlatNGN.cap_max.values/lonlatNGN.cap_max.sum()).mean('location').to_dataframe()
tempDPiw = (tempDPi*lonlatNGN.cap_max.values/lonlatNGN.cap_max.sum()).mean('location').to_dataframe()

In [16]:
b=np.array([273.3]*len(tempiw.values))

In [ ]:
r = rLF(tempiw.t2m.values,tempiw.index.map(tempDPiw.d2m).values)

In [18]:
def SDD(T):
    a = 7.5
    b = np.array([273.3]*len(T))
    b[T<0] = (265.5+240.7)/2
    return(6.1079*10**((a*T)/(b+T)))

def rLF(T,TD):
    return(100*SDD(TD)/SDD(T))

def makevars(temp,wind,tempDP,prec,rad,lons,lats,weight):
    tempi = temp.interp(coords={"longitude":xr.DataArray(lons.values,dims='location'),
                              "latitude":xr.DataArray(lats.values,dims='location')},
                      method="nearest")
    windi = wind.interp(coords={"longitude":xr.DataArray(lons.values,dims='location'),
                              "latitude":xr.DataArray(lats.values,dims='location')},
                      method="nearest")
    tempDPi = tempDP.interp(coords={"longitude":xr.DataArray(lons.values,dims='location'),
                              "latitude":xr.DataArray(lats.values,dims='location')},
                      method="nearest")
    preci = prec.interp(coords={"longitude":xr.DataArray(lons.values,dims='location'),
                              "latitude":xr.DataArray(lats.values,dims='location')},
                      method="nearest")
    radi = rad.interp(coords={"longitude":xr.DataArray(lons.values,dims='location'),
                              "latitude":xr.DataArray(lats.values,dims='location')},
                      method="nearest")
    # weigh by weights and sum up
    tempiw = (tempi*weight.values/weight.sum()).mean('location').to_dataframe()
    windiw = (windi*weight.values/weight.sum()).mean('location').to_dataframe()
    tempDPiw = (tempDPi*weight.values/weight.sum()).mean('location').to_dataframe()
    preciw = (preci*weight.values/weight.sum()).mean('location').to_dataframe()
    radiw = (radi*weight.values/weight.sum()).mean('location').to_dataframe()
    # calculate humidity
    humiw = pd.DataFrame(rLF(tempiw.t2m.values,tempiw.index.map(tempDPiw.d2m).values),index=tempiw.index)
    # merge
    res = pd.concat([tempiw,windiw,humiw,preciw,radiw])
    res.columns = ['temp','wind','hum','prec','rad']
    return(res)

In [19]:
varOUTN = makevars(temp,wind,tempDP,prec,rad,lonlatNGN.Longitude,lonlatNGN.Latitude,lonlatNGN.cap_max)

In [21]:
tgN = tempNGN.to_dataframe()
tgS = tempNGS.to_dataframe()

### model

In [34]:
outFUELN = outagesNGN[:'2021-02-14']
outFUELS = outagesNGS[:'2021-02-14']